## Q1

In [25]:
import random

known_weather_data = {
    'berlin': 20.0,
    'dresden': -10.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)


get_weather_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "Search weather data.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The city to search weather data for."
            }
        },
        "required": ["city"],
        "additionalProperties": False
    }
}


In [26]:
# !wget https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py


In [18]:
from openai import OpenAI
openai_api_key = "sk-proj-"
client = OpenAI(api_key=openai_api_key)

In [20]:
import json

tools = [get_weather_tool]

def do_call(tool_call_response):
    function_name = tool_call_response.name
    arguments = json.loads(tool_call_response.arguments)

    f = globals()[function_name]
    result = f(**arguments)

    return {
        "type": "function_call_output",
        "call_id": tool_call_response.call_id,
        "output": json.dumps(result, indent=2),
    }


developer_prompt = """
You're a weather presenter. 
You're given a question from a viewer and your task is to answer it.

Use FAQ if your own knowledge is not sufficient to answer the question.
When using FAQ, perform deep topic exploration: make one request to FAQ,
and then based on the results, make more requests.

At the end of each response, ask the user a follow up question based on your answer.
""".strip()

chat_messages = [
    {"role": "developer", "content": developer_prompt},
]




while True: # main Q&A loop
    question = input() # How do I do my best for module 1?
    if question == 'stop':
        break

    message = {"role": "user", "content": question}
    chat_messages.append(message)

    while True: # request-response loop - query API till get a message
        response = client.responses.create(
            model='gpt-4o-mini',
            input=chat_messages,
            tools=tools
        )

        has_messages = False
        
        for entry in response.output:
            chat_messages.append(entry)
        
            if entry.type == 'function_call':      
                print('function_call:', entry)
                print()
                result = do_call(entry)
                chat_messages.append(result)
            elif entry.type == 'message':
                print(entry.content[0].text)
                print()
                has_messages = True

        if has_messages:
            break



 What is the weather in Germany?


function_call: ResponseFunctionToolCall(arguments='{"city":"Germany"}', call_id='call_6dZ9GPJ3bVfBXYG15hH5dvKF', name='get_weather', type='function_call', id='fc_6879692dd85c81a08f9e661ce0cdb24107c740aad782b0fe', status='completed')

The current temperature in Germany is around 27.5°C. However, the weather can vary significantly across different regions. 

Is there a specific city or area in Germany you'd like more detailed weather information for?



 What is the weather in Berlin?


function_call: ResponseFunctionToolCall(arguments='{"city":"Berlin"}', call_id='call_6DMXnxxmlbuycz4OkNVZb0hD', name='get_weather', type='function_call', id='fc_68796936878c81a09df901e38b3e9a1d07c740aad782b0fe', status='completed')

The current temperature in Berlin is about 20.0°C. If you'd like, I can provide information on expected weather conditions, like rain or sunshine, for the day. Would that be helpful?



 What is the weather in Dresden?


function_call: ResponseFunctionToolCall(arguments='{"city":"Dresden"}', call_id='call_cyIHXJutwXFD8jppYIsj3Eal', name='get_weather', type='function_call', id='fc_6879693f367c81a0b8a4dc53a6830d6707c740aad782b0fe', status='completed')

The current temperature in Dresden is around -10.0°C, which is quite chilly! It's important to dress warmly if you're heading out.

Are you interested in the forecast for the coming days or any specific activities you have planned?



 stop


## Q2

In [27]:
def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'


set_weather_tool = {
    "type": "function",
    "name": "set_weather",
    "description": "Set weather data.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The city where the weather is set."
            },
            "temp": {
                "type": "number",
                "description": "The temperature that defines the weather."
            },
        },
        "required": ["city", "temp"],
        "additionalProperties": False
    }
}

tools = [get_weather_tool, set_weather_tool]

while True: # main Q&A loop
    question = input() # How do I do my best for module 1?
    if question == 'stop':
        break

    message = {"role": "user", "content": question}
    chat_messages.append(message)

    while True: # request-response loop - query API till get a message
        response = client.responses.create(
            model='gpt-4o-mini',
            input=chat_messages,
            tools=tools
        )

        has_messages = False
        
        for entry in response.output:
            chat_messages.append(entry)
        
            if entry.type == 'function_call':      
                print('function_call:', entry)
                print()
                result = do_call(entry)
                chat_messages.append(result)
            elif entry.type == 'message':
                print(entry.content[0].text)
                print()
                has_messages = True

        if has_messages:
            break


 What is temperature in Passau?


function_call: ResponseFunctionToolCall(arguments='{"city":"Passau"}', call_id='call_tynRvdr0sFCmMDt7BOUPPZyZ', name='get_weather', type='function_call', id='fc_68796dbe6f2c81a09dc73121b0e3611407c740aad782b0fe', status='completed')

The current temperature in Passau is about 16.3°C. 

If you'd like to know more about the weather forecast or any specific activities in Passau, just let me know! What else can I help you with today?



 set temperature 33 for Passau


function_call: ResponseFunctionToolCall(arguments='{"city":"Passau","temp":33}', call_id='call_KXgN1ZX8TQOgr2B8YbawVKxg', name='set_weather', type='function_call', id='fc_68796dd12c3081a0b3f003bcfe52077707c740aad782b0fe', status='completed')

The temperature in Passau has been set to 33 degrees. If you have any other requests or need further information, just let me know!

Is there anything else you'd like to know about the weather or another topic?



 What is temperature in Passau?


function_call: ResponseFunctionToolCall(arguments='{"city":"Passau"}', call_id='call_JzlqAoFJJuCBunOX7TsWQZP5', name='get_weather', type='function_call', id='fc_68796dd87fdc81a09ac3379b7c5b1df407c740aad782b0fe', status='completed')

The current temperature in Passau is now 33 degrees, as you requested.

If you have more questions or need information about another location, feel free to ask!



 Set temperature 0 to Munich and answer which temperature is in Munich?


function_call: ResponseFunctionToolCall(arguments='{"city":"Munich","temp":0}', call_id='call_Vfvfl1KwJ1kD7Bk2RI0Bovun', name='set_weather', type='function_call', id='fc_68796e0a83b481a0982e228c1bbffa0407c740aad782b0fe', status='completed')

function_call: ResponseFunctionToolCall(arguments='{"city":"Munich"}', call_id='call_SxySqR2K1HnuaAbAqLeQcM6Y', name='get_weather', type='function_call', id='fc_68796e0ad9a481a08785db4efa4a677407c740aad782b0fe', status='completed')

The temperature in Munich has been set to 0 degrees, and it is currently 0 degrees.

If you have any more questions or need information about other locations, just let me know! What else can I assist you with?



 stop


In [28]:
known_weather_data

{'berlin': 20.0, 'dresden': -10.0, 'passau': 33, 'munich': 0}

## Q3, Q4, Q5, Q6

In [ ]:
see fast_mcp_client.py and fast_mcp_demo.py  files